In [2]:
import pandas as pd

df = pd.read_csv('MELD.Raw/dev_sent_emo.csv',encoding='utf-8')
df['Utterance'] = df.Utterance.str.replace('',"'")
# df['gender'] = df.Speaker.apply(lambda x: 'male' if x == 'Ross' or x == 'Joey' or x == 'Chandler' else 'female')
df = df.sample(frac=0.2,ignore_index=True)
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,461,You tell them to wait!,Ross,anger,negative,42,1,5,1,"00:21:11,937","00:21:14,230"
1,778,"Oh, fine! Take his side!",Phoebe,anger,negative,72,18,5,11,"00:12:13,774","00:12:16,234"
2,451,"Okay Joey, first of all Kash Ford is not peopl...",Rachel,anger,negative,40,6,8,5,"00:06:56,874","00:06:58,750"
3,792,Seriously. What?!,Rachel,surprise,negative,74,10,8,3,"00:10:18,909","00:10:20,827"
4,861,No. Have you seen David?,Max,neutral,neutral,80,1,1,10,"00:12:04,807","00:12:06,683"
...,...,...,...,...,...,...,...,...,...,...,...
217,771,"Yeah, well, y'know maybe you just need to try ...",Phoebe,anger,negative,72,11,5,11,"00:11:43,911","00:11:47,914"
218,105,It's a witness not a perp. And no one talks li...,Gary,anger,negative,10,4,5,20,"0:07:07,593","0:07:10,096"
219,815,"So it's kinda like, you're, you know.",Chandler,neutral,neutral,76,12,3,1,"00:17:00,769","00:17:08,860"
220,445,I thought I was a complete idiot.,Rachel,surprise,positive,40,0,8,5,"00:06:37,605","00:06:40,857"


In [3]:
from nltk.stem import WordNetLemmatizer
import re


def preprocess(text):
    t = text.lower()
    t = re.sub('\d+',r'',t)
    t = re.sub(r'\W+',r' ',t)
    return t

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])
                 .strip() for txt in df['Utterance']]

In [4]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categories=[df.Emotion.unique()])
labels = ohe.fit_transform(df.Emotion.to_numpy().reshape(-1,1)).toarray()

In [32]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertModel
import numpy as np

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=7)

# Prepare your dataset for text classification
train_texts = list(df.prepro.values)
train_labels = list(labels)

# Tokenize input texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# Convert labels to tensors
train_labels = tf.convert_to_tensor(train_labels)

train_inputs = {
    'input_ids': np.array(train_encodings['input_ids']),
    'token_type_ids': np.array(train_encodings['token_type_ids']),
    'attention_mask': np.array(train_encodings['attention_mask'])
}

# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    train_inputs,
    train_labels
)).shuffle(len(train_inputs)).batch(16,drop_remainder=True)  # Adjust batch size as needed

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Fine-tune the model
model.fit(train_dataset, epochs=3, verbose=1)  # Adjust epochs as needed


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
13/13 [==============================] - 67s 3s/step - loss: 1.7582 - accuracy: 0.3413
Epoch 2/3
13/13 [==============================] - 40s 3s/step - loss: 1.5708 - accuracy: 0.4423
Epoch 3/3
13/13 [==============================] - 39s 3s/step - loss: 1.4293 - accuracy: 0.4808


In [33]:
blue = tokenizer('i am sad',return_tensors='tf')
blue

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 1045, 2572, 6517,  102]])>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]])>}

In [37]:
pred = model(blue)
pred

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
array([[ 0.26373643, -0.0307746 ,  1.4964006 ,  0.60248244, -1.3415707 ,
        -0.9906973 , -0.2134671 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [54]:
model.